# Imports

In [1]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [2]:
np.random.seed(42)
initial_params = np.random.random([48])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [3]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [4]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label in [0, 2, 4, 6, 8]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [5]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [6]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [7]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [8]:
device = qml.device("default.qubit", wires=8)

In [9]:
def unitary(params, wire1, wire2):
    # qml.RZ(0, wires=wire1)
    qml.RY(params[0], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[1], wires=wire2)
    # qml.RZ(0, wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[2], wires=wire2)
    qml.CNOT(wires=[wire1, wire2])
    qml.RY(params[3], wires=wire2)
    qml.CNOT(wires=[wire2, wire1])
    # qml.RZ(0, wires=wire1)
    qml.RY(params[4], wires=wire1)
    # qml.RZ(0, wires=wire1)
    # qml.RZ(0, wires=wire2)
    qml.RY(params[5], wires=wire2)
    # qml.RZ(0, wires=wire2)

In [10]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    unitary(params[0:6], 0, 1)
    unitary(params[6:12], 2, 3)
    unitary(params[12:18], 4, 5)
    unitary(params[18:24], 6, 7)

    # Second layer
    unitary(params[24:30], 1, 2)
    unitary(params[36:42], 5, 6)

    # Third layer
    unitary(params[42:48], 2, 5)

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [11]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643
 0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184
 0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041
 0.60754485 0.17052412 0.06505159 0.94888554 0.96563203 0.80839735
 0.30461377 0.09767211 0.68423303 0.44015249 0.12203823 0.49517691
 0.03438852 0.9093204  0.25877998 0.66252228 0.31171108 0.52006802]

Example features: [-2.42751457 -1.03578763  0.37408318 -1.91125471  1.59210347  3.16607676
  2.69330527 -1.2733277 ]

Expectation value: 0.31828450144765463

 0: ──RY(-2.43)──RY(0.375)───╭X─────────────╭C─────────────╭X──RY(0.156)──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤     
 1: ──RY(-1.04)──RY(0.951)───╰C──RY(0.732)──╰X──RY(0.599)──╰C──RY(0.156)───RY(0.456)───╭

# Accuracy test definition

In [12]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [13]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.5512605042016807
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.5786314525810324
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.6312124849939976
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.6362545018007203
best accuracy so far!
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.6605042016806723
best accuracy so far!
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.6561824729891956
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.6737094837935174
best accuracy so far!
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.6631452581032413
epoch 7

epoch 341
epoch 342
epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.7195678271308523
test_accuracies: [0.6737094837935174, 0.6631452581032413, 0.6830732292917168, 0.6470588235294117, 0.6957983193277311, 0.6801920768307324, 0.695078031212485, 0.6912364945978391, 0.7034813925570228, 0.7008403361344537, 0.7142857142857143, 0.6986794717887155, 0.7054021608643457, 0.7114045618247299, 0.7166866746698679, 0.7135654261704683, 0.7166866746698679, 0.7032412965186075, 0.7166866746698679, 0.7188475390156062, 0.7272509003601441, 0.7224489795918367, 0.7262905162064826, 0.7114045618247299, 0.7253301320528212, 0.7236494597839136, 0.7082833133253301, 0.7188475390156062, 0.7255702280912365, 0.7195678271308523]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7325330132052821
best accuracy so far!
test_accuracies: [0.6631452581032413, 0.6830732292917168, 0.6470588235294117, 0.6957983193277311, 0.680

epoch 451
epoch 452
epoch 453
epoch 454
epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.6965186074429772
test_accuracies: [0.6986794717887155, 0.7054021608643457, 0.7114045618247299, 0.7166866746698679, 0.7135654261704683, 0.7166866746698679, 0.7032412965186075, 0.7166866746698679, 0.7188475390156062, 0.7272509003601441, 0.7224489795918367, 0.7262905162064826, 0.7114045618247299, 0.7253301320528212, 0.7236494597839136, 0.7082833133253301, 0.7188475390156062, 0.7255702280912365, 0.7195678271308523, 0.7325330132052821, 0.7253301320528212, 0.7205282112845138, 0.7255702280912365, 0.7178871548619448, 0.7250900360144057, 0.7296518607442977, 0.7205282112845138, 0.7164465786314526, 0.7224489795918367, 0.6965186074429772]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7207683073229292
test_accuracies: [0.7054021608643457, 0.7114045618247299, 0.7166866746698679, 0.7135654261704683, 0.7166866746698679, 0.7032

epoch 561
epoch 562
epoch 563
epoch 564
epoch 565
epoch 566
epoch 567
epoch 568
epoch 569
epoch 570
accuracy: 0.7183673469387755
test_accuracies: [0.7262905162064826, 0.7114045618247299, 0.7253301320528212, 0.7236494597839136, 0.7082833133253301, 0.7188475390156062, 0.7255702280912365, 0.7195678271308523, 0.7325330132052821, 0.7253301320528212, 0.7205282112845138, 0.7255702280912365, 0.7178871548619448, 0.7250900360144057, 0.7296518607442977, 0.7205282112845138, 0.7164465786314526, 0.7224489795918367, 0.6965186074429772, 0.7207683073229292, 0.7298919567827131, 0.7174069627851141, 0.7298919567827131, 0.7224489795918367, 0.7111644657863145, 0.7277310924369749, 0.7286914765906363, 0.7171668667466986, 0.7104441776710684, 0.7183673469387755]
epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.729171668667467
test_accuracies: [0.7114045618247299, 0.7253301320528212, 0.7236494597839136, 0.7082833133253301, 0.7188475390156062, 0.72557

epoch 671
epoch 672
epoch 673
epoch 674
epoch 675
epoch 676
epoch 677
epoch 678
epoch 679
epoch 680
accuracy: 0.7214885954381753
test_accuracies: [0.7255702280912365, 0.7178871548619448, 0.7250900360144057, 0.7296518607442977, 0.7205282112845138, 0.7164465786314526, 0.7224489795918367, 0.6965186074429772, 0.7207683073229292, 0.7298919567827131, 0.7174069627851141, 0.7298919567827131, 0.7224489795918367, 0.7111644657863145, 0.7277310924369749, 0.7286914765906363, 0.7171668667466986, 0.7104441776710684, 0.7183673469387755, 0.729171668667467, 0.7234093637454981, 0.7164465786314526, 0.7332533013205282, 0.720048019207683, 0.7118847539015607, 0.7274909963985594, 0.7222088835534214, 0.7207683073229292, 0.7087635054021608, 0.7214885954381753]
epoch 681
epoch 682
epoch 683
epoch 684
epoch 685
epoch 686
epoch 687
epoch 688
epoch 689
epoch 690
accuracy: 0.7039615846338536
test_accuracies: [0.7178871548619448, 0.7250900360144057, 0.7296518607442977, 0.7205282112845138, 0.7164465786314526, 0.722448

epoch 781
epoch 782
epoch 783
epoch 784
epoch 785
epoch 786
epoch 787
epoch 788
epoch 789
epoch 790
accuracy: 0.7231692677070829
test_accuracies: [0.7298919567827131, 0.7224489795918367, 0.7111644657863145, 0.7277310924369749, 0.7286914765906363, 0.7171668667466986, 0.7104441776710684, 0.7183673469387755, 0.729171668667467, 0.7234093637454981, 0.7164465786314526, 0.7332533013205282, 0.720048019207683, 0.7118847539015607, 0.7274909963985594, 0.7222088835534214, 0.7207683073229292, 0.7087635054021608, 0.7214885954381753, 0.7039615846338536, 0.7195678271308523, 0.7349339735894358, 0.7301320528211285, 0.726530612244898, 0.7334933973589436, 0.7070828331332533, 0.7039615846338536, 0.7217286914765906, 0.7178871548619448, 0.7231692677070829]
epoch 791
epoch 792
epoch 793
epoch 794
epoch 795
epoch 796
epoch 797
epoch 798
epoch 799
epoch 800
accuracy: 0.7334933973589436
test_accuracies: [0.7224489795918367, 0.7111644657863145, 0.7277310924369749, 0.7286914765906363, 0.7171668667466986, 0.7104441

epoch 892
epoch 893
epoch 894
epoch 895
epoch 896
epoch 897
epoch 898
epoch 899
epoch 900
accuracy: 0.7294117647058824
test_accuracies: [0.7332533013205282, 0.720048019207683, 0.7118847539015607, 0.7274909963985594, 0.7222088835534214, 0.7207683073229292, 0.7087635054021608, 0.7214885954381753, 0.7039615846338536, 0.7195678271308523, 0.7349339735894358, 0.7301320528211285, 0.726530612244898, 0.7334933973589436, 0.7070828331332533, 0.7039615846338536, 0.7217286914765906, 0.7178871548619448, 0.7231692677070829, 0.7334933973589436, 0.7226890756302521, 0.7152460984393758, 0.7150060024009603, 0.7181272509003602, 0.7289315726290516, 0.7315726290516207, 0.7248499399759905, 0.7217286914765906, 0.7025210084033613, 0.7294117647058824]
epoch 901
epoch 902
epoch 903
epoch 904
epoch 905
epoch 906
epoch 907
epoch 908
epoch 909
epoch 910
accuracy: 0.7296518607442977
test_accuracies: [0.720048019207683, 0.7118847539015607, 0.7274909963985594, 0.7222088835534214, 0.7207683073229292, 0.7087635054021608,

In [14]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 2.4786283   0.33560636  1.93194864  1.30794859  0.15601864 -0.4394482
 -2.52021475  0.73629933 -0.06622457  0.07828772 -0.39035535  0.96990985
  0.62073053 -1.60429488 -0.23217154 -0.18700347  0.30424224  0.29294727
  0.78795894  0.45493012  1.07096237  0.02498426  1.12352496  0.36636184
 -0.13937273  0.37423611 -0.69177892  0.78771186  0.59241457 -0.12845917
  0.60754485  0.17052412  0.06505159  0.94888554  0.96563203  0.80839735
  0.0728046   0.92905242  1.24369291  1.60384547 -0.38602599  0.49517691
 -0.14052106  0.40125618  0.87039268  0.81886503  0.31171108  0.57446961]


# Testing

In [15]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.727047619047619


In [16]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 6.710109710693359
training time: 5705.9700355529785
test time: 129.34613966941833
total time: 5842.02628493309
